In [10]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [8]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
def calcCov(x, y):
    return (np.sum((x-np.mean(x))*(y-np.mean(y))))/(x.shape[0]-1)

def covMat(data):
    cov_mat = np.zeros(data.shape)
    for row in covMat:
        print(row)

In [41]:
iris.data.shape

(150, 4)

In [39]:
x = np.array(iris_df['sepal length (cm)'])
y = np.array(iris_df['sepal width (cm)'])
z = np.array(iris_df['petal length (cm)'])
calcCov(x, y)

np.float64(-0.0421511111111111)

In [38]:
X = np.array([1, 2, 3, 4, 5])
Y = np.array([1, 2, 3, 4, 5])

# Compute covariance matrix
cov_matrix = np.cov(x, y, rowvar=False, bias=True)

print(cov_matrix)

[[ 0.68112222 -0.04215111]
 [-0.04215111  0.18871289]]
